In [2]:
import pandas as pd
import math

In [3]:
player_time = pd.read_csv('player_time.csv')
player_ratings = pd.read_csv('player_ratings.csv')

In [20]:
player_time.head()


,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,60,409,0,63,142,1,186,168,0,...,62,345,6,351,100,0,77,613,1,125
2,0,120,546,0,283,622,4,645,330,0,...,543,684,12,805,200,0,210,815,5,323
3,0,180,683,1,314,927,9,1202,430,0,...,842,958,16,1135,300,0,210,1290,8,527
4,0,240,956,1,485,1264,11,1583,530,0,...,1048,1500,26,1842,400,0,210,1431,9,589


In [19]:
player_ratings.head()

,account_id,total_wins,total_matches,trueskill_mu,trueskill_sigma
0,236579,14,24,27.868035,5.212361
1,-343,1,1,26.544163,8.065475
2,-1217,1,1,26.521103,8.114989
3,-1227,1,1,27.248025,8.092217
4,-1284,0,1,22.931016,8.092224


In [6]:
players = pd.read_csv('players.csv')

In [18]:
players.head()

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,unit_order_glyph,unit_order_eject_item_from_stash,unit_order_cast_rune,unit_order_ping_ability,unit_order_move_to_direction,unit_order_patrol,unit_order_vector_target_position,unit_order_radar,unit_order_set_item_combine_lock,unit_order_continue
0,0,0,86,0,3261,10960,347,362,9,3,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,51,1,2954,17760,494,659,13,3,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,83,2,110,12195,350,385,0,4,...,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2,11,3,1179,22505,599,605,8,4,...,1.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,3,67,4,3307,23825,613,762,20,3,...,3.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
teamfights = pd.read_csv('teamfights.csv')
teamfights.head()

,match_id,start,end,last_death,deaths
0,0,220,252,237,3
1,0,429,475,460,3
2,0,900,936,921,3
3,0,1284,1328,1313,3
4,0,1614,1666,1651,5


In [9]:
teamfights_players = pd.read_csv('teamfights_players.csv')

To predict the kill-death ratio for a team fight, we will need to first establish the class variable by melting the dataframe so each row represents a teamfight for one team (two rows per fight). 
Before modifying the teamfight_players dataset to form the class variable we will need to extract some other pieces of information first, specifically xp delta and a column to estimate whether the hero was in the fight or not based on gold and xp delta. 

In [10]:
teamfights_players = teamfights_players[0:1000]#cutting data to test feasibility first.

In [11]:
#remove damage, buyback columns
teamfights_players = teamfights_players.drop(['damage', 'buybacks'], axis = 1)

In [12]:
#add 'active' column
def isActive(gold, xp, slot):#A player is 'active' during (not necessarily present in) a team fight if the gain gold and xp
    if gold==0 and xp==0:
        return -1
    else:
        return slot
    
teamfights_players['active'] = teamfights_players.apply(
    lambda x: isActive(x['xp_end'] - x['xp_start'], x['gold_delta'], x['player_slot']), axis = 1)

In [16]:
#unpivot match_id, player_slot, fight_id as index with sum of deaths and participating player slot matrix
#add fight index
teamfights_players['fight_id'] = pd.Series(teamfights_players.index.values/10).apply(math.floor)

In [13]:
teamfights_players.head()

,match_id,player_slot,deaths,gold_delta,xp_end,xp_start,active
0,0,0,0,173,536,314,0
1,0,1,1,0,1583,1418,1
2,0,2,0,0,391,391,-1
3,0,3,0,123,1775,1419,3
4,0,4,0,336,1267,983,4


In [45]:
#calculate Kill-Death ratio
#sum deaths for each team, for each fight
#add team index
def findTeam(player_slot):
    if player_slot < 100:
        return True
    else:
        return False
KDR = teamfights_players[['match_id', 'player_slot', 'deaths', 'fight_id']]
KDR['radiant'] = KDR.apply(lambda x: findTeam(x['player_slot']), axis = 1)
KDR.head()

C:\Users\R\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,match_id,player_slot,deaths,fight_id,radiant
0,0,0,0,0,True
1,0,1,1,0,True
2,0,2,0,0,True
3,0,3,0,0,True
4,0,4,0,0,True


In [54]:
deathCount = KDR.groupby('fight_id')['deaths'].sum()


In [60]:
deathCount[0:20]

fight_id
0      3
1      3
2      3
3      3
4      5
5      3
6      6
7      5
8      5
9      4
10     4
11     3
12     3
13     3
14     3
15     3
16     3
17     9
18    11
19     3
Name: deaths, dtype: int64

In [47]:
#create table with columns for each player slot
active = teamfights_players.pivot_table(columns = ['player_slot'], index = ['match_id', 'fight_id'])
active = active['active']
active.head()

player_slot        0    1    2    3    4    128  129  130  131  132
match_id fight_id                                                  
0        0           0    1   -1    3    4  128  129  130   -1  132
         1           0    1    2    3    4  128  129  130  131  132
         2           0    1    2    3    4  128   -1  130  131  132
         3           0    1    2    3    4  128  129  130  131  132
         4           0    1    2    3    4  128  129  130  131  132

In [48]:
#take the players in a fight, the time and match
#return the sum of gold, xp, and last hits for each team
def pre_stats(t, m, p):
    measures = ['gold', 'xp', 'lh']
    pre_stats = player_time.loc[((t - 60) < player_time['times']) & (player_time['times'] <= t) & (player_time['match_id'] == m)]
    players_rad = [p[i] for i in range(5) if p[i] >= 0]
    players_dir = [p[i] for i in range(5, 10) if p[i] >= 0]#a list of dire players, where -1 indicates the player wasn't active
        
    radiant = {measures[m] : #dictionary of radiant team statistics for last hits, gold and experience
               sum([pre_stats[measures[m] + '_t_' + str(p[i])] for i in range(5) if p[i] >= 0])
               for m in range(3)}
    dire = {measures[m] : # Example dictionary:{'gold': 134, 'xp':10536, 'lh':235}
            sum([pre_stats[measures[m] + '_t_' + str(p[i])] for i in range(5, 10) if p[i] >= 0]) 
            for m in range(3)}
    
    #sum([pre_stats[gold + str(p[i])] for i in range(5) if p[i] >= 0])
    #xp_rad = sum([pre_stats[xp + str(p[i])] for i in range(5) if p[i] >= 0]))
    #astHits_rad = sum([pre_stats[lastHits + str(p[i])] for i in range(5) if p[i] >= 0]))
    
    #gold_dir = sum([pre_stats[gold + str(p[i])] for i in range(5, 10) if p[i] >= 0]))
    #xp_dir = sum([pre_stats[xp + str(p[i])] for i in range(5, 10) if p[i] >= 0]))
    #lastHits_dir = sum([pre_stats[lastHits + str(p[i])] for i in range(5, 10) if p[i] >= 0]))
    
    return {'radiant': radiant, 'dire': dire}

In [ ]:
#Create a dataframe for cumulative team stats
#gold


In [50]:
#now we need to add the above to a dataframe. oof
#First we need to make the dataframe. 
#It needs columns that can derive start_time, and players
teamfights = pd.read_csv('teamfights.csv')
teamfights['fight_id'] = pd.Series(teamfights.index.values)
#teamfights = teamfights.set_index(['match_id'], append = True)
teamfights = teamfights.set_index(['fight_id'], append = True)
#active.index = active.index.droplevel(0)

#teamfights.merge(active, how = 'left', left_index = True, right_index = True)
#teamfights.merge(active, how = 'inner', on = ['fight_id'])

,,match_id,start,end,last_death,deaths
,fight_id,,,,,
0,0,0,220,252,237,3
1,1,0,429,475,460,3
2,2,0,900,936,921,3
3,3,0,1284,1328,1313,3
4,4,0,1614,1666,1651,5
5,5,0,1672,1709,1694,3
6,6,0,1734,1783,1768,6
7,7,0,1818,1867,1852,5
8,8,0,1863,1912,1897,5


Once this is done, the dataset can be grouped to show one row per team per teamfight. The columns would include the total kills, total XP, and one one column per player slot indicating whether they were present in the fight or not based on their XP and gold delta. 

In [ ]:
#Calculate 